<a href="https://colab.research.google.com/github/micheldion/colab-dev01/blob/main/depenses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
from google.colab import drive
import pandas as pd 
import math
import glob

drive.mount('/content/gdrive')

TPS = 0.05
TVQ = 0.09975
FISCAL_MONTH_INDEX = 4

def fiscalMonthClosure(index):
   def c(month):
      return month -index if month > index else (12-index) + month
   return c

currentFiscalMonth = fiscalMonthClosure(FISCAL_MONTH_INDEX)

def fiscalYear(date):
  return str(date.year)+"-"+str(date.year+1)  if date.month > 4 else str(date.year  -1)+"-"+str(date.year)

def round_half_up(n):
  multiplier = 10 ** 2
  return math.floor(n*multiplier + 0.5) / multiplier

def computeTPS (subTotal):
  return round_half_up(subTotal * TPS)

def computeTVQ (subTotal):
  return round_half_up(subTotal * TVQ)  

def taxePeriod(month):
    if   month in [11,12,1]:
      return 1
    elif month in [2,3,4]:
      return 2
    elif month in [5,6,7]:
      return 3
    elif month in [8,9,10]:
      return 4
    else:
      raise Exception("Not in range "+str(month))


def process(glob, folder, isExpense=True):
  data_files = glob.glob(folder)
  V = [i.split("/")[-1].split(".pdf")[0] for i in data_files]
  #for i in V:
    #print(i)
  df = pd.DataFrame(V,columns=['file']) 
  df[['PaymentDate','InvoiceDate','SubTotal','TPS','TVQ','MODE','Desc']] = df['file'].str.split("_",expand=True,)
  df.PaymentDate = pd.to_datetime(df.PaymentDate, errors="raise")
  df.InvoiceDate = pd.to_datetime(df.InvoiceDate, errors="raise")
  df.SubTotal = pd.to_numeric(df.SubTotal, errors="raise")
  df.TPS = pd.to_numeric(df.TPS, errors="raise")
  df.TVQ = pd.to_numeric(df.TVQ, errors="raise")
  
  #df['Day'] =  df['InvoiceDate'].dt.day
  #df['Month'] =  df['InvoiceDate'].dt.month
  
  df['Total'] = df.SubTotal + df.TPS + df.TVQ
  df['ComputedTPS'] = df.SubTotal.map(computeTPS)
  df['ComputedTVQ'] = df.SubTotal.map(computeTVQ)
  df['AssertTPS'] = (df.ComputedTPS == df.TPS) | (df.TPS == 0)
  df['AssertTVQ'] = (df.ComputedTVQ == df.TVQ) | (df.TVQ == 0)
  df['OK'] = (df.AssertTPS) & (df.AssertTVQ)

  FiscalDate = "PaymentDate" if isExpense else "InvoiceDate"

  df["FiscalMonth"] = df[FiscalDate].dt.month.map(currentFiscalMonth)
  df['FiscalYear'] =  df[FiscalDate].map(fiscalYear)
  df['TaxePeriod'] =  df[FiscalDate].dt.month.map(taxePeriod)


  df2 = df[['PaymentDate','InvoiceDate','FiscalYear','FiscalMonth', 'TaxePeriod','MODE', 'Desc','SubTotal','TPS','TVQ','Total','file']]
  return df2.set_index(df2.PaymentDate).sort_index(),df


def totalByMonth(glob, folder):
  t = process(glob, folder)[0]
  t = t.groupby(pd.Grouper(freq='M'))['Total','TPS','TVQ'].sum() 
  t['year'] = t.index.year
  t['Month'] = t.index.month
 
  t.index =t.index.month.map(currentFiscalMonth)
  t.index.names = ['FiscalMonth']
  return t

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
a = process(glob, "/content/gdrive/My Drive/datazentrik/déboursés/2*/*.*")[0]
a
#a[a['FiscalMonth']==5]

In [ ]:
sommaire  = totalByMonth(glob, "/content/gdrive/My Drive/datazentrik/déboursés/2*/*.*")
sommaire

In [ ]:
t = process(glob, "/content/gdrive/My Drive/datazentrik/recettes/2*/*.*", False)[0]
t

In [ ]:
def build_visa_payment_dates(glob, folder):
  data_files = glob.glob(folder)
  V = [i.split("/")[-1].split(".pdf")[0] for i in data_files]
  df = pd.DataFrame(V,columns=['File']) 
  print(V)
  df[['PaymentDate','InvoiceDate','BeginDate','Desc']] = df['File'].str.split("_",expand=True,)
  df.PaymentDate = pd.to_datetime(df.PaymentDate, errors="raise")
  df.InvoiceDate = pd.to_datetime(df.InvoiceDate, errors="raise")
  df.BeginDate = pd.to_datetime(df.BeginDate, errors="raise")
  visa = df[['PaymentDate','InvoiceDate','BeginDate','File']]
  r = pd.date_range(start=visa.BeginDate.min(), end=visa.InvoiceDate.max())
  dates = pd.DataFrame(r, columns=['InvoiceDate'])
  dates = dates.merge(visa, left_on='InvoiceDate', right_on='InvoiceDate',how='outer').set_index("InvoiceDate").sort_index()
  dates['PaymentDate']= dates.PaymentDate.bfill(axis=0)
  dates['File']= dates.File.bfill(axis=0)
  dates = dates.reset_index()[['InvoiceDate','PaymentDate']]
  return dates

payment_dates = build_visa_payment_dates(glob, "/content/gdrive/My Drive/datazentrik/déboursés/visa/*_visa.pdf")  

payment_dates


In [ ]:
aa= a.drop(columns=['PaymentDate']).reset_index()
test = payment_dates.merge(aa.reset_index(), left_on='InvoiceDate', right_on='InvoiceDate',how='inner')
#test.groupby(['DatePaiement2'])[['SubTotal','TPS','TVQ','Total']].sum()
#test[test['DatePaiement'] == '2020-11-12']
test.set_index("PaymentDate_x").sort_index()
test

In [ ]:
a